In [3]:
import os,re
import requests as rq
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy

In [4]:
data = rq.get("https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles=List_of_hip_hop_musicians").json()
list_markup = data["query"]["pages"]["140853"]["revisions"][0]["*"]
link_regex = r"\* \[\[(.+?)[\(\]]"

rappers = re.findall(link_regex, list_markup)

In [20]:
with open("rappers.txt", 'w') as f:
    for r in rappers:
        f.write(r.encode('utf-8') + "\n")

In [7]:
import spotipy.oauth2 as oauth2

# When an error occurs saying the token expired, re-run this cell.
# Tokens are 1 hour I think.
client_id = 'e3344de9f0454df280cc34cdba040bf7'
client_secret = 'da09e980b2fb4f11a68a3c00a2669a7d'

credentials = oauth2.SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [9]:
def get_albums(rapper):
    results = spotify.search(q='artist:%s' % rapper,type='artist')
    
    artist_id = results['artists']['items'][0]['uri']
    
    albums = spotify.artist_albums(artist_id)
    
    # For some reason duplicates some albums. Wrap as a set
    return set([album['name'] for album in albums['items']])

In [10]:
def get_album_artists(album_name):
    results = spotify.search(q='album:%s' % album_name,type='album')
    
    album_id = results['albums']['items'][0]['uri']
    
    tracks = spotify.album_tracks(album_id)
    return set([artist['name']  
            for track in tracks['items']
            for artist in track['artists']])

In [11]:
def get_collaborators(rapper):
    """Returns a dictionary of album:artists entry"""
    
    albums = get_albums(rapper)
    
    return {album:list(get_album_artists(album)) for album in albums}

In [12]:
get_collaborators("Drake")

{u"If You're Reading This It's Too Late": [u'Lil Wayne',
  u'PARTYNEXTDOOR',
  u'Drake',
  u'Travis Scott'],
 u'More Life': [u'Kanye West',
  u'Travis Scott',
  u'Quavo',
  u'Giggs',
  u'Jorja Smith',
  u'2 Chainz',
  u'Drake',
  u'Black Coffee',
  u'Young Thug',
  u'PARTYNEXTDOOR',
  u'Sampha'],
 u'Nothing Was The Same': [u'Big Sean',
  u'Detail',
  u'Drake',
  u'2 Chainz',
  u'Majid Jordan',
  u'Sampha',
  u'JAY Z',
  u'Jhene Aiko'],
 u'Nothing Was The Same (Deluxe)': [u'Big Sean',
  u'Detail',
  u'Drake',
  u'2 Chainz',
  u'Majid Jordan',
  u'Sampha',
  u'JAY Z',
  u'Jhene Aiko'],
 u'Scorpion': [u'Ty Dolla $ign',
  u'Static Major',
  u'JAY Z',
  u'Drake',
  u'Michael Jackson'],
 u'Views': [u'Kyla',
  u'WizKid',
  u'Drake',
  u'Rihanna',
  u'Future',
  u'PARTYNEXTDOOR',
  u'Pimp C',
  u'dvsn'],
 u'What A Time To Be Alive': [u'Future', u'Drake']}